In [3]:
import numpy as np
import astropy.units as u
import time
from datetime import datetime
today = int(datetime.today().strftime('%Y%m%d'))
import poppy
import skimage
from skimage.registration import phase_cross_correlation
import copy
from importlib import reload

import magpyx
from magpyx.utils import ImageStream
import purepyindi
from purepyindi import INDIClient
client0 = purepyindi.INDIClient('localhost', 7624)
client0.start()

import purepyindi2
from purepyindi2 import IndiClient
client = IndiClient()
client.connect()
client.get_properties()


from math_module import xp, xcipy, np_array
import utils
import fsm_utils
import cam_utils
from imshows import imshow1, imshow2, imshow3
from matplotlib.patches import Circle

v_bias = np.array([[50,50,50]]).T

wavelength = 633e-9
fl = 500e-3
fsm_pupil_diam = 12e-3
as_per_lamD = (wavelength/fsm_pupil_diam*u.radian).to(u.arcsec)
print(as_per_lamD)

pxscl_lamD = 3.45e-6 / (fl * wavelength/fsm_pupil_diam)
print(pxscl_lamD)


10.880468529534333 arcsec
0.13080568720379146


INFO:purepyindi2.transports:Connected to localhost:7624


In [12]:
fsm = ImageStream('fsm')
fsm.write(v_bias)

In [18]:
v_tt = fsm_utils.get_fsm_volts(tip=0*as_per_lamD, tilt=0.5*as_per_lamD)
volts = xp.array([v_bias, v_tt])
utils.save_fits('tt_mod.fits', volts)

Displacements: A = 5.00e-06 m, 5.16e-06 m, 4.84e-06 m. 
Delta Voltages: A = 50.00, B = 51.58, C = 48.42. 
Saved data to:  tt_mod.fits


In [19]:
volts

array([[[50.    ],
        [50.    ],
        [50.    ]],

       [[50.    ],
        [51.5825],
        [48.4175]]])

In [11]:
def set_fsm_mod_amp(amp, client, delay=0.25):
    client.wait_for_properties(['fsmModulator.amp'])
    client['fsmModulator.amp.target'] = amp
    time.sleep(delay)

def set_fsm_mod_rate(freq, client, delay=0.25):
    client.wait_for_properties(['fsmModulator.frequency'])
    client['fsmModulator.frequency.target'] = freq
    time.sleep(delay)

def start_fsm_mod(client, delay=0.25):
    client.wait_for_properties(['fsmModulator.trigger', 'fsmModulator.modulating'])
    client['fsmModulator.trigger.toggle'] = purepyindi.SwitchState.OFF
    time.sleep(delay)
    client['fsmModulator.modulating.toggle'] = purepyindi.SwitchState.ON
    time.sleep(delay)

def stop_fsm_mod(client, delay=0.25):
    client.wait_for_properties(['fsmModulator.trigger', 'fsmModulator.modulating', 'fsmModulator.zero'])
    client['fsmModulator.modulating.toggle'] = purepyindi.SwitchState.OFF
    time.sleep(delay)
    client['fsmModulator.trigger.toggle'] = purepyindi.SwitchState.ON
    time.sleep(delay)
    client['fsmModulator.zero.request'] = purepyindi.SwitchState.ON

In [ ]:
set_fsm_mod_amp(1, client0)
set_fsm_mod_rate(2, client0)


In [20]:
start_fsm_mod(client0)

In [21]:
stop_fsm_mod(client0)

In [22]:
fsm.write(v_bias)

In [28]:
v_tt = fsm_utils.get_fsm_volts(tip=0*as_per_lamD, tilt=1*as_per_lamD)
volts = [v_bias, v_tt]
freq = 3
try:
    print('Modulating FSM ...')
    i = 0
    while True:
        fsm.write(volts[i%2])
        i += 1 
        time.sleep(1/freq/2)
except KeyboardInterrupt:
    print('FSM modulation stopped!')
    fsm.write(v_bias)


Displacements: A = 5.00e-06 m, 5.32e-06 m, 4.68e-06 m. 
Delta Voltages: A = 50.00, B = 53.16, C = 46.84. 
Modulating FSM ...
FSM modulation stopped!


In [43]:
fsm.write(0*v_bias)